# Homework

### 環境の準備

In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [11]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [2]:
import random
random.seed(0)

In [14]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [15]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

# 2. 文字起こしデータの活用
## 講義内容をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

In [20]:
from sentence_transformers import SentenceTransformer

#!nvidia-smi
emb_model = SentenceTransformer('intfloat/multilingual-e5-base', trust_remote_code=True) # to avoid OOM
# emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
# emb_model.max_seq_length = 8192
emb_model.max_seq_length = 1024

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [108]:
# input prompt

system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
#question =  "日本で一番面積が大きな都道府県はどこですか？"
question =  "超新星残骸で加速されたヘリウムよりも重いイオンを検出するにはどうすれば良いですか？"
#question =  "物語で姫は正体はなんでしたか？"
#question =  "2025 年 5 月に行われた将棋の第 38 期竜王戦 5 組ランキング戦決勝戦の対戦カードはなんですか？"
#question =  "2005 年に愛知で開催された万博でマスコットであるミャクミャクのデザインはどのように選ばれましたか？"
#question =  "2025 年に大阪で開催された万博でマスコットであるミャクミャクの赤と青の由来をそれぞれ教えてください？"


In [ ]:
with open("./sample_data/question2_arf.txt", "r") as f:
  raw_writedown = f.read()

# ドキュメントを用意する。
#documents = [text.strip() for text in raw_writedown.split("。")]
documents = [text.strip() for text in raw_writedown.split(".")] # English
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[0])

ドキュメントサイズ:  1
ドキュメントの例: 
 超新星残骸（SNR）は、銀河宇宙線の主要な加速源と見なされています。最近のX線研究では、いくつかのSNRの近くにある高密度のガスから中性鉄のKα線放射が検出され、この現象は、SNR内で加速された準相対論的粒子によってガス中の鉄原子がK殻電離を受けた結果として最も適切に解釈されています。この論文では、現在不明であるSNR内で加速された粒子の組成を制限する新しい方法を提案します。エネルギーの高い重イオンが標的原子と衝突すると、その強いクーロン場によって標的原子の内殻電子が同時に複数個放出されることが容易に起こります。この結果、特性X線線エネルギーがシフトし、特徴的なスペクトル構造が形成されます。このような構造が中性鉄Kα線において検出されると、粒子電離シナリオを強く支持することになり、さらに加速された粒子中に重イオンが含まれているという直接的な証拠を提供します。我々は、原子衝突データを利用して、さまざまな入射イオンによるFe Kα線の構造をモデル化しました。


In [ ]:
# `query:` quwery にプレフィックスを追加
query = "query: " + question

# エンコード（バッチ処理で処理）
query_embeddings = emb_model.encode([query])
document_embeddings = emb_model.encode(["passage: " + doc for doc in documents])

# Cosine similarity計算
scores = (query_embeddings @ document_embeddings.T) * 100

# 結果表示
print(scores.tolist())

[[89.05923461914062]]


In [122]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 89.05923461914062)
超新星残骸（SNR）は、銀河宇宙線の主要な加速源と見なされています。最近のX線研究では、いくつかのSNRの近くにある高密度のガスから中性鉄のKα線放射が検出され、この現象は、SNR内で加速された準相対論的粒子によってガス中の鉄原子がK殻電離を受けた結果として最も適切に解釈されています。この論文では、現在不明であるSNR内で加速された粒子の組成を制限する新しい方法を提案します。エネルギーの高い重イオンが標的原子と衝突すると、その強いクーロン場によって標的原子の内殻電子が同時に複数個放出されることが容易に起こります。この結果、特性X線線エネルギーがシフトし、特徴的なスペクトル構造が形成されます。このような構造が中性鉄Kα線において検出されると、粒子電離シナリオを強く支持することになり、さらに加速された粒子中に重イオンが含まれているという直接的な証拠を提供します。我々は、原子衝突データを利用して、さまざまな入射イオンによるFe Kα線の構造をモデル化しました。 




In [123]:
response = generate_output(question, system_prompt)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


超新星残骸（Supernova Remnant）で加速されたイオンの検出には、特に重いイオンの検出に適した観測方法や技術が必要です。以下は、重いイオンの検出に役立つ観測方法の例です。

1. X線望遠鏡：超新星残骸のX線スペクトルを観測することで、加速されたイオンの存在を検出することができます。X線望遠鏡は、イオンのX線放射を検出するために使用されます。
2. γ線望遠鏡：γ線望遠鏡は、加速されたイオンのγ線放射を検出するために使用されます。γ線は、イオンの崩壊過程で放出される高エネルギーのγ線です。
3. Radio望遠鏡：Radio望遠鏡は、加速されたイオンのRadio波を検出するために使用されます。Radio波は、イオンの電磁波として放出されます。
4. Spectroscopy：Spectroscopyは、加速されたイオンのスペクトルを


In [124]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question_RAG =  f"[参考資料]\n{references}\n\n[質問] {question}"
response_RAG = generate_output(question_RAG, system_prompt)

print(response_RAG)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


超新星残骸(SNR)で加速された粒子の組成を制限する新しい方法を提案する論文において、原子衝突データを利用して、さまざまな入射イオンによるFe Kα線の構造をモデル化しています。そこで、超新星残骸で加速されたヘリウムよりも重いイオンを検出するには、以下の方法が提案されます。

まず、X線研究で検出された中性鉄Kα線のスペクトル構造を分析する必要があります。特に、Fe Kα線の構造が、重イオンによる電離シナリオを支持する場合、粒子電離シナリオを強く支持することになります。

次に、原子衝突データを利用して、さまざまな入射イオンによるFe Kα線の構造をモデル化する必要があります。モデル化されたFe Kα線の構造を比較することで、超新星残骸で加速されたヘリウムよりも重いイオンが含まれているかどうかを判断することができます。

最後に


In [125]:
# 答えの比較
print(f"RAG なし: {response}")
print(f"RAG あり: {response_RAG}")

RAG なし: 超新星残骸（Supernova Remnant）で加速されたイオンの検出には、特に重いイオンの検出に適した観測方法や技術が必要です。以下は、重いイオンの検出に役立つ観測方法の例です。

1. X線望遠鏡：超新星残骸のX線スペクトルを観測することで、加速されたイオンの存在を検出することができます。X線望遠鏡は、イオンのX線放射を検出するために使用されます。
2. γ線望遠鏡：γ線望遠鏡は、加速されたイオンのγ線放射を検出するために使用されます。γ線は、イオンの崩壊過程で放出される高エネルギーのγ線です。
3. Radio望遠鏡：Radio望遠鏡は、加速されたイオンのRadio波を検出するために使用されます。Radio波は、イオンの電磁波として放出されます。
4. Spectroscopy：Spectroscopyは、加速されたイオンのスペクトルを
RAG あり: 超新星残骸(SNR)で加速された粒子の組成を制限する新しい方法を提案する論文において、原子衝突データを利用して、さまざまな入射イオンによるFe Kα線の構造をモデル化しています。そこで、超新星残骸で加速されたヘリウムよりも重いイオンを検出するには、以下の方法が提案されます。

まず、X線研究で検出された中性鉄Kα線のスペクトル構造を分析する必要があります。特に、Fe Kα線の構造が、重イオンによる電離シナリオを支持する場合、粒子電離シナリオを強く支持することになります。

次に、原子衝突データを利用して、さまざまな入射イオンによるFe Kα線の構造をモデル化する必要があります。モデル化されたFe Kα線の構造を比較することで、超新星残骸で加速されたヘリウムよりも重いイオンが含まれているかどうかを判断することができます。

最後に


In [118]:
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
messages = [
    {"role": "system", "content": "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"},
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] {question}"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [119]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

参考資料に基づいて、超新星残骸(SNRs)で加速されたヘリウムよりも重いイオンを検出するには、以下の方法が提案されます。

1. Kα線の観測：加速されたイオンが衝突したターゲット原子に内殻電子を同時に放出させることで、Kα線のエネルギーがずれることがあります。このような構造を検出することで、イオンの存在を直接証明することができます。
2. KαLiピークの観測：Kα線の観測結果に基づいて、KαLiピークの強度がプロジェクトイルの荷電状態と運動エネルギーに依存していることが示されます。より大きな荷電状態を持つプロジェクトイルほど、KαLiピークが強くなるため、イオンの荷電状態を推定することができます。
3. 多重イオン化の影響の評価：加速されたイオンが衝突したターゲット原子に内殻電子を同時に放出させることで、Kα線のエネルギーがずれることがあります。このような構造を検出することで、イオンの存在を証明することができます。

以上のように、超新星残骸(SNRs)で加速されたヘリウムよりも重いイオンを検出するには、Kα線の観測、KαLiピークの観測、多重イオン化の影響の評価などの方法を組み合わせることが必要です。
